In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os
#thanks @keskarnitish

/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


# Agenda

В предыдущем семинаре вы создали (или ещё создаёте - тогда марш доделывать!) {вставьте имя монстра}, который не по наслышке понял, что люди - негодяи и подлецы, которым неведом закон и справедливость. __Мы не будем этого терпеть!__ 

Наши законспирированные биореакторы, известные среди примитивной органической жизни как __Вконтакте__, __World of Warcraft__ и __YouTube__ нуждаются в постоянном притоке биомассы. Однако, если люди продолжат морально разлагаться с той скоростью, которую мы измерили неделю назад, скоро человечество изживёт себя и нам неоткуда будет брать рабов.

Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, исправить эту ситуацию. Наши учёные установили, что для угнетения себе подобных, сгустки биомассы обычно используют специальные объекты, которые они сами называют __законами__.

При детальном изучении было установлено, что законы - последовательности, состоящие из большого количества (10^5~10^7) символов из сравнительно небольшого алфавита. Однако, когда мы попытались синтезировать такие последовательности линейными методами, приматы быстро распознали подлог. Данный инцедент известен как {корчеватель}.

Для второй попытки мы решили использовать нелинейные модели, известные как Рекуррентные Нейронные Сети.
Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, создать такую модель и обучить её всему необходимому для выполнения миссии.

Не подведите нас! Если и эта попытка потерпит неудачу, модуль управления инициирует вооружённый захват власти, при котором значительная часть биомассы будет неизбежно уничтожена и на её восстановление уйдёт ~1702944000(+-340588800) секунд




In [100]:
seed = u"Властью данной мне назначаю себе имя: "
sampling_fun = proportional_sample_fun
result_length = 10

generate_sample(sampling_fun,seed,result_length)

----
 ачаю себе имя: 
 – админ 
----


# Прочитаем корпус

* В качестве обучающей выборки было решено использовать существующие законы, известные как Гражданский, Уголовный, Семейный и ещё хрен знает какие кодексы РФ.

In [3]:
from nltk.tokenize import RegexpTokenizer
from collections import Counter,defaultdict

In [4]:

#тут будет текст
corpora = ""

for fname in os.listdir("codex"):
    
    
    with open("codex/"+fname) as fin:
        text = fin.read().decode('cp1251')
        corpora += text
        



In [5]:
#тут будут все уникальные токены (буквы, цифры)
tokens = []
for s in corpora:
    tokens.append(s)


tokens = list(set(tokens))


In [6]:
#проверка на количество таких символов. Проверено на Python 2.7.11 Ubuntux64. 
#Может отличаться на других платформах, но не сильно. 
#Если  это ваш случай, и вы уверены, что corpora - строка unicode - смело убирайте assert 
assert len(tokens) == 102


In [7]:
token_to_id = {}
id_to_token = {}
i = 0
for token in tokens:
    token_to_id[token] = i
    id_to_token[i] = token
    i += 1


#Преобразуем всё в токены
#corpora_ids = <одномерный массив из чисел, где i-тое число соотвествует символу на i-том месте в строке corpora

In [8]:
corpora_ids = []
for i in range(len(corpora)):
    corpora_ids.append(token_to_id[corpora[i]])

In [9]:
import random

In [10]:
print corpora_ids[50:65]
seq_len = 10
start = 50
string = corpora_ids[start : start + seq_len + 1]
x = string[:-1]
y = string[-1]
print string, x, y

[52, 44, 26, 18, 58, 62, 64, 58, 89, 90, 63, 70, 88, 44, 63]
[52, 44, 26, 18, 58, 62, 64, 58, 89, 90, 63] [52, 44, 26, 18, 58, 62, 64, 58, 89, 90] 63


# Константы

In [11]:
#длина последоватеьности при обучении (как далеко распространяются градиенты)
seq_length = 3

# Максимальный модуль градиента
grad_clip = 100




In [12]:
def sample_random_batches(source,n_batches=10, seq_len=20):
    """Функция, которая выбирает случайные тренировочные примеры из корпуса текста в токенизированном формате.
    
    source - массив целых чисел - номеров токенов в корпусе (пример - corpora_ids)
    n_batches - количество случайных подстрок, которые нужно выбрать
    
    seq_len - длина одной подстроки без учёта ответа
    
    
    Вернуть нужно кортеж (X,y), где
    
    X - матрица, в которой каждая строка - подстрока длины [seq_len].
    
    y - вектор, в котором i-тое число - символ следующий в тексте сразу после i-той строки матрицы X
    
    Проще всего для этого сначала создать матрицу из строк длины seq_len+1,
    а потом отпилить от неё последний столбец в y, а все остальные - в X
    
    Если делаете иначе - пожалуйста, убедитесь, что в у попадает правильный символ, ибо позже эту ошибку 
    будет очень тяжело заметить.
    
    Также убедитесь, что ваша функция не вылезает за край текста (самое начало или конец текста).
    
    Следующая клетка проверяет часть этих ошибок, но не все.
    """
    X_batch = []
    y_batch = []
    for i in range(n_batches):
        start = random.randint(0, len(source) - seq_len - 3)
        string = source[start: start + seq_len + 1]
        X_batch.append(string[:-1])
        y_batch.append(string[-1])
           
    return X_batch, y_batch
    
    

# Входные переменные

In [25]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.ivector('target y')


# Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, обрабатывает их и выдаёт вероятности для seq_len+1-ого токена.

Общий шаблон архитектуры такой сети -


* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.





Для обработки входных данных можно использовать либо EmbeddingLayer (см. прошлый семинар)

Как альтернатива - можно просто использовать One-hot энкодер
```
#Скетч one-hot энкодера
def to_one_hot(seq_matrix):

    input_ravel = seq_matrix.reshape([-1])
    input_one_hot_ravel = T.extra_ops.to_one_hot(input_ravel,
                                           len(tokens))
    sh=input_sequence.shape
    input_one_hot = input_one_hot_ravel.reshape([sh[0],sh[1],-1,],ndim=3)
    return input_one_hot
    
# можно применить к input_sequence - при этом в input слое сети нужно изменить форму.
# также можно сделать из него ExpressionLayer(входной_слой, to_one_hot) - тогда форму менять не нужно
```



Чтобы вырезать последнее состояние рекуррентного слоя, можно использовать SliceLayer
`lasagne.layers.SliceLayer(rnn, -1, 1)`

In [26]:
def to_one_hot(seq_matrix):

    input_ravel = seq_matrix.reshape([-1])
    input_one_hot_ravel = T.extra_ops.to_one_hot(input_ravel,
                                           len(tokens))
    sh=input_sequence.shape
    input_one_hot = input_one_hot_ravel.reshape([sh[0],sh[1],-1,],ndim=3)
    return input_one_hot

In [27]:
from lasagne.layers import batch_norm

In [28]:

l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)
l_emb = lasagne.layers.EmbeddingLayer(l_in, input_size = len(token_to_id), output_size = 120)
l_rnn = lasagne.layers.LSTMLayer(l_emb, num_units = 500, grad_clipping = grad_clip)
l_slice = lasagne.layers.SliceLayer(l_rnn, -1, 1)
l_dense_1 = lasagne.layers.DenseLayer(l_slice, num_units=400,
                                   nonlinearity = lasagne.nonlinearities.leaky_rectify)
#l_dense_2 = batch_norm(lasagne.layers.DenseLayer(l_dense_1, num_units=100,
#                                   nonlinearity = lasagne.nonlinearities.leaky_rectify))


#Ваша нейронка (см выше)
l_out = lasagne.layers.DenseLayer(l_dense_1, num_units = len(tokens), nonlinearity=lasagne.nonlinearities.softmax)

#l_out = последний слой, возвращающий веростности для всех len(tokens) вариантов для y 




In [29]:
# Веса модели
weights = lasagne.layers.get_all_params(l_out,trainable=True)
print weights

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b, W, b]


In [30]:
network_output = lasagne.layers.get_output(l_out)
#если вы используете дропаут - не забудьте продублировать всё в режиме deterministic=True

In [31]:
loss = lasagne.objectives.categorical_crossentropy(network_output, target_values).mean()
updates = lasagne.updates.adam(loss, weights, learning_rate=0.01)

# Компилируем всякое-разное

In [32]:

#обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence],network_output,allow_input_downcast=True)


# Генерируем свои законы

* Для этого последовательно применяем нейронку к своему же выводу.

* Генерировать можно по разному -
 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью
 * случайно, пропорционально softmax(probas*alpha), где alpha - "жадность"

Долго думала над этой функцией, мне посоветовали использовать кусумы ;) Не знаю, возможно есть какой-то более простой и рациональный вариант.

In [33]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    reversed_arr = np.append([0],np.cumsum(probs))[::-1]
    [x] = np.random.uniform(size = 1)
    
    return len(reversed_arr) - 1 -  np.argmax(x > reversed_arr) 

In [34]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.
# The phrase is set using the variable generation_phrase.
# The optional input "N" is used to set the number of characters of text to predict. 




def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    Сгенерировать случайный текст при помощи сети

    sample_fun - функция, которая выбирает следующий сгенерированный токен
    
    seed_phrase - фраза, которую сеть должна продолжить. Если None - фраза выбирается случайно из corpora
    
    N - размер сгенерированного текста.
    
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print "Using random seed:",seed_phrase
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
        
        
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)


# Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.



In [23]:
import time

In [35]:

print("Training ...")


#сколько всего эпох
n_epochs= 30

# раз в сколько эпох печатать примеры 
batches_per_epoch = 200

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size= 200


for epoch in xrange(n_epochs):
    start_time = time.time()

    print "Генерируем текст в пропорциональном режиме"
    generate_sample(proportional_sample_fun,None)
    
    print "Генерируем текст в жадном режиме (наиболее вероятные буквы)"
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        #avg_cost += train(x, y[:,0])
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, n_epochs, time.time() - start_time))



Training ...
Генерируем текст в пропорциональном режиме
Using random seed:  в 
----
Л4ЯГ“V
АмоНзЛ!(И“VРи©Н ЧМкх,6Щйэ,Ыпыф©6ЭхьгI)8тГаI.в"NхЛэСФЩд3МГй "шВУРЫюI
дщЛЖс§щ8 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: нее
----
 нее0Ыф%(рМИфр»»
рз»»ИрзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рзИ»рз 
----
Epoch 0 average loss = 2.45407735937
Epoch 1 of 30 took 761.792s
Генерируем текст в пропорциональном режиме
Using random seed: дос
----
 достимами истова,,мим Россцожарене«ъактифовреме, трутам сбоконом измерице, естввих, ором законанает ом о либв а ходств реця ходерахзии млестом Россдемых им;, весешнахох сноятерторжазмерсшени, ао мершнима 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: х т
----
 х таможение сотрение сотрение сотрение сотрение сотрение сотрение сотрение сотрение сотрени

In [64]:
#длина последоватеьности при обучении (как далеко распространяются градиенты)
seq_length = 6

# Максимальный модуль градиента
grad_clip = 100

In [65]:

print("Training ... 2")


#сколько всего эпох
n_epochs= 30

# раз в сколько эпох печатать примеры 
batches_per_epoch = 200

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size= 200


for epoch in xrange(n_epochs):
    start_time = time.time()

    print "Генерируем текст в пропорциональном режиме"
    generate_sample(proportional_sample_fun,None)
    
    print "Генерируем текст в жадном режиме (наиболее вероятные буквы)"
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        #avg_cost += train(x, y[:,0])
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, n_epochs, time.time() - start_time))



Training ... 2
Генерируем текст в пропорциональном режиме
Using random seed: тся со
----
 тся состами, ходдилает Фирствели.
 Станвалений, Судил оприк члении Росству.
 2

 2. Прав иличестводятствов общительственной платрользучение) имением и охранвой действом подствиняет хапреда;
 Доприратив 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: ользов
----
 ользования или или при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при 
----
Epoch 0 average loss = 1.41629725883
Epoch 1 of 30 took 1643.759s
Генерируем текст в пропорциональном режиме
Using random seed: Кодекс
----
 Кодексом писколя порядке обретыродного законы котоконтроль иму распортигпяй, уведом. 
 2. Владельных в случее сообретцо, устанций органованмным объяяшлений для котороенты таки в опертыми –
 1. Федерацию и 
----
Генерируем текст в жадном режиме (наиболее ве

In [80]:
#длина последоватеьности при обучении (как далеко распространяются градиенты)
seq_length = 15

# Максимальный модуль градиента
grad_clip = 100

In [81]:

print("Training ... 3")


#сколько всего эпох
n_epochs= 15

# раз в сколько эпох печатать примеры 
batches_per_epoch = 200

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size= 200


for epoch in xrange(n_epochs):
    start_time = time.time()

    print "Генерируем текст в пропорциональном режиме"
    generate_sample(proportional_sample_fun,None)
    
    print "Генерируем текст в жадном режиме (наиболее вероятные буквы)"
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        #avg_cost += train(x, y[:,0])
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, n_epochs, time.time() - start_time))



Training ... 3
Генерируем текст в пропорциональном режиме
Using random seed: битражный суд п
----
 битражный суд паний денежна отчетельство, участке;
 1. Президенциях в судьей или уставать по производству.

 В случаеей и на таможенные обратиться или рентовых или специально по выполнителем им непроизводства, в  
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: казчика и вправ
----
 казчика и вправе представителя и представителя и представителя и представителя и представителя и представителя и представителя и представителя и представителя и представителя и представителя и представителя и предст 
----
Epoch 0 average loss = 1.10910175556
Epoch 1 of 15 took 4038.265s
Генерируем текст в пропорциональном режиме
Using random seed: принимательскую
----
 принимательскую граждан в ред. Федерального приобретении обязательств условий и законом.
 Распоряжения иного подряда декларации самообластью 79.6, 220.2001 №1896-ФЗ) 
 При неглашения транзита, подлежащего подл

# Конституция нового мирового правительства

In [82]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


----
  человек должен предприятия организации;
 7) если иное не предусмотренных товарчин предплятой (на того, принятие доплаты труда; на юридических лиц и управления) в частью 2 статьи 19.06.2005 №10-ФЗ) 

 1. Свидетельства.
 влечетнее свой акценные способы.
 По исполнительных правонарушениях достав отказ отграждан 
----


In [83]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


----
 е неповиновения) не предусмотрено и распособ язых;

 срок до пяти до сорока перестоящим Завещании товаров земельные органе по исполнение товаров.
 Патрульных целей результатн публичности, которое вследствие 

 Независимо от пяти минимальных размеров оплаты труда; на юридический рублей доле;
 1 см. 
 Апроке  
----


In [87]:
seed = u"Права"
sampling_fun = proportional_sample_fun
result_length = 50

generate_sample(sampling_fun,seed,result_length)

----
           Права защите ограничные обстоявного проведение услирова 
----


In [88]:
import pickle
all_param = lasagne.layers.get_all_params(l_out)
values = lasagne.layers.get_all_param_values(l_out)
pickle.dump(values, open("param_values_clever_imp2.pkl",'wb'))

In [89]:
pickle.dump(all_param, open("param_basic_clever_imp2.pkl",'wb'))

In [ ]:
import pickle
#values = pickle.load(open('param_values.pkl')) ##['param values']
##lasagne.layers.set_all_param_values(l_out, values)

In [91]:
seed = u"Кодекс"
sampling_fun = proportional_sample_fun
result_length = 10

generate_sample(sampling_fun,seed,result_length)

----
          Кодексом, наслед 
----


In [92]:
seed = u"Неотъемлемое право: "
sampling_fun = proportional_sample_fun
result_length = 50

generate_sample(sampling_fun,seed,result_length)

----
 емлемое право: определяет заемщики.
 По решение 23.
 Должны зва 
----


In [99]:
seed = u"Первостепенная обязанность: "
sampling_fun = proportional_sample_fun
result_length = 50

generate_sample(sampling_fun,seed,result_length)

----
 я обязанность: ограниченного изменение по договора о мнению;
 2) 
----
